In [17]:
import torch
import torch.nn as nn

In [11]:
x = torch.tensor(1., requires_grad = True)
w = torch.tensor(2., requires_grad = True)
b = torch.tensor(3., requires_grad = True)

In [13]:
y = w*x + b
y.backward()

In [37]:
print(x.grad)
print(w.grad)
print(b.grad)

None
tensor(1.)
tensor(1.)


In [42]:
x = torch.randn(10, 3)
y = torch.randn(10, 2)
# 实例化模型
linear = nn.Linear(3, 2)
print(linear.weight)
print(linear.bias)
# 实例化目标函数、优化器
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

Parameter containing:
tensor([[ 0.3153, -0.0369,  0.5598],
        [ 0.0326,  0.5248, -0.0057]], requires_grad=True)
Parameter containing:
tensor([0.5059, 0.0343], requires_grad=True)


In [43]:
y_pre = linear(x)
loss = criterion(y, y_pre)

print('before train .loss:', loss.item())

before train .loss: 32.376705169677734


In [44]:
# 开始训练
loss.backward()
print('dw:', linear.weight.grad)
print('db:', linear.bias.grad)

print(linear.weight)
print(linear.bias)

dw: tensor([[ 3.3697, -0.6791,  8.8961],
        [ 0.1031, 14.8054, 11.4115]])
db: tensor([13.3412,  3.5310])
Parameter containing:
tensor([[ 0.3153, -0.0369,  0.5598],
        [ 0.0326,  0.5248, -0.0057]], requires_grad=True)
Parameter containing:
tensor([0.5059, 0.0343], requires_grad=True)


In [45]:
optimizer.step()
print(linear.weight)
print(linear.bias)

Parameter containing:
tensor([[ 0.2816, -0.0302,  0.4708],
        [ 0.0316,  0.3768, -0.1199]], requires_grad=True)
Parameter containing:
tensor([ 0.3725, -0.0010], requires_grad=True)


In [46]:
y_pre = linear(x)
loss = criterion(y, y_pre)
print('after 1 train .loss:', loss.item())

after 1 train .loss: 26.88092803955078


#2层前馈神经网络 预测MNIST

In [25]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transfrorms
import torch.utils.data as data

# 超参数
input_size = 784
hidden_size = 500
num_classes = 10
num_epoch = 5
batch_size = 100
learning_rate = 1e-4

# 数据加载
# dataset
train_data = torchvision.datasets.MNIST(
    root='pytorch-Learning/data',
    train=True,
    transform=transfrorms.ToTensor(),
    download=True
)
test_data = torchvision.datasets.MNIST(
    root='pytorch-Learning/data',
    train=False,
    transform=transfrorms.ToTensor()
)
# dataloader
train_loader = data.DataLoader(
    dataset=train_data,
    batch_size=batch_size,  # 批大小，用于每次训练的样本数量
    shuffle=True  # 每步迭代时，要打乱dataset的顺序来取出batch_size个样本
)
test_loader = data.DataLoader(
    dataset=test_data,
    batch_size=batch_size,  # 批大小，用于每次训练的样本数量
    shuffle=False  # 每步迭代时，不要打乱dataset的顺序来取出batch_size个样本
)


# 定义2层全连接模型
class twoLayerNeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, ouput_size):
        super(twoLayerNeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, ouput_size)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x


model = twoLayerNeuralNet(input_size, hidden_size, num_classes)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# 开始训练:
# 训练集有60000，每个训练的批大小为100，于是每一个epoch的迭代（step）次数为600次：
# 即data_loader的大小(total_step)
total_step = len(train_loader)
for epoch in range(num_epoch):
    for i, (images, labels) in enumerate(train_loader):
        images = images.reshape(-1, 28 * 28)  # 把图片flat化
        # 前向传播，计算损失
        outputs = model(images)
        loss = criterion(outputs, labels)
        # 后向传播，更新权值
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch + 1, num_epoch, i + 1, total_step, loss.item()))

# 开始测试 不计算梯度（内存效率）
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28 * 28)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))


Epoch [1/5], Step [100/600], Loss: 1.2281


Epoch [1/5], Step [200/600], Loss: 0.6860


Epoch [1/5], Step [300/600], Loss: 0.4523


Epoch [1/5], Step [400/600], Loss: 0.5192


Epoch [1/5], Step [500/600], Loss: 0.3145


...

Epoch [5/5], Step [300/600], Loss: 0.2222


Epoch [5/5], Step [400/600], Loss: 0.2119


Epoch [5/5], Step [500/600], Loss: 0.1944


Epoch [5/5], Step [600/600], Loss: 0.1821


Accuracy of the network on the 10000 test images: 94.5 %
